In [1]:
#Importing standart libraries
import seaborn as sns
import pandas as pd
import numpy as np
import os
import glob
#Additional stuff for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.cm as cm
import matplotlib.patches as mpatches

#scRNA-seq toolkit
import anndata
import scanpy as sc
import scvi

#Libraries for working with matrices and predictions evaluations
from scipy.sparse import csr_matrix
import numba
import pynndescent
from scvi.model.utils import mde

#Muting annoying things
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Importing some other stuff from our module
from utils_and_functions import *

/home/z.starinnov/miniforge3/envs/scvi_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model training and implementation

In [ ]:
## Read subsetted MERFISH dataset with hypothalamic neurons for scVI model training (can be found on Zenodo)
adata = sc.read_h5ad('../merfish_neurons_spatial.h5ad')

In [ ]:
## Training model and creating a dataset for future projection
adata.layers['counts'] = adata.X.copy()
scvi.model.SCVI.setup_anndata(adata, layer = "counts")

model = scvi.model.SCVI(adata)
model.train(max_epochs = 400, early_stopping = True,batch_size=100)

adata.obsm['X_scVI'] = model.get_latent_representation()
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)

sc.pp.neighbors(adata, use_rep = 'X_scVI')
sc.tl.tsne(adata)

adata.write_h5ad('../model_path/model_adata.h5ad')
model.save('../model_path/', overwrite=True)

In [ ]:
## Read input file
adata_input = sc.read('../input_path/')

In [ ]:
## Perform standatd sc-preprocessing and clustering on sc-dataset
adata_input = do_tsne(adata_input, resolution = 1)

In [ ]:
## Read model weigths path and model dataset 
adata_hypomap = sc.read('../model_path/model_adata.h5ad')
model_path = '../model_path/'

In [ ]:
## Feed each cluster of adata_input into a model and calculate labeling with PyNNdescent
out = []
for i in np.unique(adata_input.obs.clusters):
    adata_query = adata_input[adata_input.obs.clusters == i]
    adata_query.layers['counts'] = adata_query.X.copy()
    vars = scvi.model.SCVI.prepare_query_anndata(adata_query, model_path,return_reference_var_names = True)
    del adata_query.varm
    scvi.model.SCVI.prepare_query_anndata(adata_query, model_path)
    vae_query = scvi.model.SCVI.load_query_data(
    adata_query,
    model_path)
    vae_query.train(max_epochs=400, plan_kwargs=dict(weight_decay=0.0))
    adata_query.obsm["X_scVI"] = vae_query.get_latent_representation()
    
    ## calculate UMAP based on hypoMAP:
    sc.pp.neighbors(adata_query, use_rep="X_scVI")
    sc.tl.leiden(adata_query)
    sc.tl.umap(adata_query)  
    
    X_train = adata_hypomap.obsm["X_scVI"]
    ref_nn_index = pynndescent.NNDescent(X_train)
    ref_nn_index.prepare()
    
    ref_emb = sc.AnnData(X_train, obs=adata_hypomap.obs)
    print(ref_emb)
    
    query_emb = sc.AnnData(vae_query.get_latent_representation())
    query_emb.obs_names = adata_query.obs_names
    ref_neighbors, ref_distances = ref_nn_index.query(query_emb.X)
    
    
    
    # convert distances to affinities
    stds = np.std(ref_distances, axis=1)
    stds = (2.0 / stds) ** 2
    stds = stds.reshape(-1, 1)
    ref_distances_tilda = np.exp(-np.true_divide(ref_distances, stds))
    weights = ref_distances_tilda / np.sum(ref_distances_tilda, axis=1, keepdims=True)
    label_keys = ['clusters']
    for l in label_keys:
        ref_cats = adata_hypomap.obs[l].astype('category').cat.codes.to_numpy()[ref_neighbors]
        p, u = weighted_prediction(weights, ref_cats)
        p = np.asarray(adata_hypomap.obs[l].astype('category').cat.categories)[p]
        query_emb.obs[l + "_pred"], query_emb.obs[l + "_uncertainty"] = p, u
    
    combined_emb = ref_emb.concatenate(query_emb)
    combined_emb.obsm["X_mde"] = mde(combined_emb.X, init="random")
    colors = [l + "_uncertainty" for l in label_keys]
    sc.pl.embedding(
        combined_emb,
        basis="X_mde",
        color=colors)
    
    colors = [l + "_pred" for l in label_keys]
    ## Uncomment if you want to see each projection on mde embedding
    ##sc.pl.embedding(combined_emb, basis="X_mde", color=colors, ncols=1, size=40)
    out.append(query_emb)

In [ ]:
## Concatanate and filter resulting predictions. Personally I prefer level of uncertainty at 0.4 but you can use harsher one
projections = sc.concat(out)
projections = projections[(projections.obs.clusters_uncertainty<0.4)

In [ ]:
##Print distiribution of projections for each sc-cluster of input data
for i in np.unique(projections.obs.clust):
    a,b = np.unique(projections[(projections.obs.clust == i)].obs.clusters_pred, return_counts=True)
    print('---------')
    print(i)
    for j in range(len(a)):
        print(a[j], round(b[j]*100/sum(b)), '%')
    print('---------')

## Projecting back on hypothtlamic slices

Metadata for MERFISH can be downloaded from 

https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#metadata/Zhuang-ABCA-1/

https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#metadata/Zhuang-ABCA-2/

Following code was taken from https://github.com/AllenInstitute/abc_atlas_access/blob/main/notebooks/zhuang_merfish_tutorial.ipynb

In [ ]:
dir_list = ['coronal1', 'coronal2']

cell = {}
for d in dir_list:
    cell[d] = pd.read_csv(f'../{d}/cell_metadata.csv')
    
    cell[d].set_index('cell_label', inplace=True)
    
    sdf = cell[d].groupby('brain_section_label')
    
    print(d,":","Number of cells = ", len(cell[d]), ", ", "Number of sections =", len(sdf))

Files for WMB annotation can be found at:

https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#metadata/WMB-taxonomy/20231215/views/

In [ ]:
cluster_colors= pd.read_csv('../WMB-taxonomy/20231215/views/cluster_to_cluster_annotation_membership_color.csv')
cluster_colors.set_index('cluster_alias', inplace=True)
cluster_details = pd.read_csv('../WMB-taxonomy/20231215/views/cluster_to_cluster_annotation_membership_pivoted.csv')
cluster_details.set_index('cluster_alias', inplace=True)

In [ ]:
cell_extended = {}

for d in dir_list:
    cell_extended[d] = cell[d].join(cluster_details, on='cluster_alias')
    cell_extended[d] = cell_extended[d].join(cluster_colors, on='cluster_alias')

In [ ]:
ccf_coordinates = {}

for d in dir_list :

    ccf_coordinates[d] = pd.read_csv(f'../{d}/ccf_coordinates.csv')
    ccf_coordinates[d].set_index('cell_label', inplace=True)
    ccf_coordinates[d].rename(columns={'x': 'x_ccf',
                                       'y': 'y_ccf',
                                       'z': 'z_ccf'},
                              inplace=True)
    
    cell_extended[d] = cell_extended[d].join(ccf_coordinates[d], how='inner')

Parcellation annotation can be downloaded at:

https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#metadata/Allen-CCF-2020/20230630/

In [ ]:
parcellation_annotation = pd.read_csv('../abc/metadata/Allen-CCF-2020/20230630/views/parcellation_to_parcellation_term_membership_acronym.csv')
parcellation_annotation.set_index('parcellation_index', inplace=True)

parcellation_color = pd.read_csv('../abc/metadata/Allen-CCF-2020/20230630/views/parcellation_to_parcellation_term_membership_color.csv')
parcellation_color.set_index('parcellation_index', inplace=True)

parcellation_color.columns = ['parcellation_%s'% x for x in  parcellation_color.columns]

In [ ]:
for d in dir_list :
    cell_extended[d] = cell_extended[d].join(parcellation_annotation, on='parcellation_index')
    cell_extended[d] = cell_extended[d].join(parcellation_color, on='parcellation_index')

In [ ]:
## Get final metadata dataframe with all required information for plottting slices
meta_all = pd.concat([cell_extended['coronal1'],cell_extended['coronal2']])

In [ ]:
## Read spatial data
adata_spatial = sc.read_h5ad('../merfish_neurons_spatial.h5ad')

In [ ]:
## Transfer labels and their colors from spatial data to metadata dataframe
meta_exp = meta_all
names=[]
for i in range(81):
    names.append(f'{i}')
#names.append('no_cluster')

meta_exp['clusters'] = 'no_cluster'

meta_exp.loc[adata_spatial.obs.index, 'clusters'] = adata_spatial.obs.clusters

clust_colors = adata_spatial.uns['clusters_colors'][0:81]

clust_dict={}
for A, B in zip(names, clust_colors):
    clust_dict[A] = B
clust_dict['no_cluster']='#D3D3D3'

meta_exp['clusters_colors']=meta_exp.clusters.map(clust_dict)

In [ ]:
## Plot sc-clusters from spatial adata on MERFISH slices
ploting_all_sections(meta_exp, 'clusters', 'clusters_colors', name = temp)